In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.plotly as py
import plotly.figure_factory as ff
import torch
from typing import List, Dict
from pprint import pprint

In [2]:
# -- Local imports
from utils.elasticsearch_utils import get_all_hits, flatten_dict, flatten_dict_keys, \
    get_hits_dsl_query, \
    get_hits_dict_query, update_fields_select_df, get_server_reports

In [ ]:
# -- Useful paths
OUT_DIR = "results/reports"
eDATA_FILTER_FILE = "configs/edata_fields.csv"
rDATA_FILTER_FILE = "configs/reporting_fields.csv"

In [ ]:
# -- Example of elasticsearch data fetch

# -- Gather all data from elasticsearch
full_edata: List[Dict] = get_all_hits()

edata: List[Dict] = get_hits_dsl_query({
    "match": {
        "args.title": {
            "query": "andrei",
            "type": "phrase"
        }
    }
})

edata: List[Dict] = get_hits_dict_query({
    "args.title": ["andrei"], 
    "args.tasks.datasets": ["mnist", "cifar10"]
})

In [ ]:
# -- Example of fetching reports from server
rdata: List[Dict] = get_server_reports(e_ids=["E1sZ62MBm5wd3rDHL-EF"])


In [4]:
# -- Key filters for reports or eData

# eDATA_FILTER_FILE Management
edata_keys = sorted(list(flatten_dict_keys(edata[0])))  # Use tu update / initialize

# Reporting columns read 
rdata_keys = sorted(list(flatten_dict_keys(rdata[0])))  # Use tu update / initialize

# -- Read /update them to file
# Read key filters and smarg_grop prop for eData 
e_select_df, e_select_k, e_select_sg = update_fields_select_df(None, edata_keys, 
                                                               update_file=eDATA_FILTER_FILE)

# Read key filters and smarg_grop prop for raw reporting data 
r_select_df, r_select_k, r_select_sg = update_fields_select_df(None, rdata_keys, 
                                                               update_file=rDATA_FILTER_FILE)

In [ ]:
# Generate table of values (remove variable sizes)
NON_TABLE_DATA = ['_score', '_index', '_type', 'best_eval', 'last_eval', 'task_info', 
                  'model_summary']

# flat_data = []
# for x in data:
#     table_data = {k: v for k, v in x.items() if k not in NON_TABLE_DATA}
#     flat_data.append(flatten_dict(table_data))
#     
# df = pd.DataFrame(flat_data)
# 
# # Select only keys in select_keys_r 
# df = df[select_keys_col]
# df.to_csv("test.csv")
# variable_args = []
# 
# for cl in df.columns:
#     if cl.startswith("args"):
#         if len(df[cl].unique()) > 1:
#             variable_args.append(cl)


In [ ]:
# Demonstrate

In [4]:
# Analize Experiment "Individual"
dfi = df[df["args.experiment"] == "individual"]
dfi["args._experiment_parameters.datasets"] = dfi["args._experiment_parameters.datasets"].apply(
    lambda x: x[0])

dfi["args.model.hidden_units"] = dfi["args.model.hidden_units"].apply(lambda x: str(x))
dfi_base = dfi.groupby(["args._experiment_parameters.datasets", "args.model.name", "args.model.hidden_units"]).max()[[
    "best_eval_acc"]]
dfi_base.index.names = ["dataset_name", "model_base_type"]
dfi_base.columns = ["base_score"]
dfi_base["best_eval_acc"] = dfi_base["best_eval_acc"].apply(lambda x: x[0])
dfi_base.to_csv("configs/base_scores")